In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !wget https://zenodo.org/records/3371780/files/audio_mono-mic.zip
# !unzip  -d /content/drive/MyDrive/GuitarJam/audio_mono-mic audio_mono-mic.zip

In [ ]:
import os
import shutil
import glob
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [ ]:
def manage_wav_files(source_dir, destination_dir, action):
    os.makedirs(destination_dir, exist_ok=True)
    wav_files = glob.glob(os.path.join(source_dir, '*.wav'))

    # Move each WAV file to the destination directory
    for wav_file in wav_files:
        destination_path = os.path.join(destination_dir, os.path.basename(wav_file))
        if action == 'copy':
            shutil.copy(wav_file, destination_path)
        elif action == 'move':
            shutil.move(wav_file, destination_path)

    print(f"{action} {len(wav_files)} files successfully.")

In [ ]:
def create_metadata_df(folder_dir: str, audio_type: str) -> pd.DataFrame:
    wav_files = glob.glob(os.path.join(folder_dir, '*.wav'))
    file_names = [f"train/{os.path.basename(wav_file)}" for wav_file in wav_files]
    df = pd.DataFrame(file_names, columns=['file_name'])
    df['audio_type'] = audio_type
    df['prompt'] = f"guitar, {audio_type}"
    return df

### Manage Comp Guitar Files

In [ ]:
comp_df = create_metadata_df("/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp/train", "accompaniment")
comp_df.to_csv("/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp/metadata.csv", index=False)
comp_df.head()

,file_name,audio_type,prompt
0,train/00_Jazz3-150-C_comp_mic.wav,accompaniment,"guitar, accompaniment"
1,train/00_Funk2-119-G_comp_mic.wav,accompaniment,"guitar, accompaniment"
2,train/00_BN2-166-Ab_comp_mic.wav,accompaniment,"guitar, accompaniment"
3,train/00_BN1-129-Eb_comp_mic.wav,accompaniment,"guitar, accompaniment"
4,train/00_BN2-131-B_comp_mic.wav,accompaniment,"guitar, accompaniment"


### Manage Solo Guitar Files

In [ ]:
manage_wav_files("/content/drive/MyDrive/GuitarJam/audio_mono-mic/solo/", "/content/drive/MyDrive/GuitarJam/audio_mono-mic/solo/train", "move")

move 180 files successfully.


In [ ]:
solo_df = create_metadata_df("/content/drive/MyDrive/GuitarJam/audio_mono-mic/solo/train", "solo")
solo_df.to_csv("/content/drive/MyDrive/GuitarJam/audio_mono-mic/solo/metadata.csv", index=False)
solo_df.head()

,file_name,audio_type,prompt
0,train/00_Funk1-97-C_solo_mic.wav,solo,"guitar, solo"
1,train/00_BN2-166-Ab_solo_mic.wav,solo,"guitar, solo"
2,train/00_Jazz2-187-F#_solo_mic.wav,solo,"guitar, solo"
3,train/00_Funk3-98-A_solo_mic.wav,solo,"guitar, solo"
4,train/00_Funk1-114-Ab_solo_mic.wav,solo,"guitar, solo"


## Build All Audio Dataset

In [ ]:
destination_dir = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/all/"
os.makedirs(destination_dir, exist_ok=True)

solo_path = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/solo/metadata.csv"
comp_path = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp/metadata.csv"

solo_df = pd.read_csv(solo_path)
comp_df = pd.read_csv(comp_path)

### Prepare Data

In [ ]:
# solo
manage_wav_files(
    "/content/drive/MyDrive/GuitarJam/audio_mono-mic/solo/train",
    os.path.join(destination_dir, "train"),
    "copy"
)

# comp
manage_wav_files(
    "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp/train",
    os.path.join(destination_dir, "train"),
    "copy"
)


copy 180 files successfully.
copy 180 files successfully.


### Prepare metadata

In [ ]:
prompt_dict = {
    "solo": "High-quality acoustic guitar solo, virtuosic performance, pure guitar melody but gentle emotional depth with dynamic techniques",
    "accompaniment": "High-quality acoustic guitar accompaniment, virtuosic performance, pure guitar melody but gentle emotional depth with dynamic techniques"
}
all_df = pd.concat([solo_df, comp_df], ignore_index=True)
all_df['prompt'] = all_df['audio_type'].map(prompt_dict)
all_df

,file_name,audio_type,prompt
0,train/00_Funk1-97-C_solo_mic.wav,solo,"High-quality acoustic guitar solo, virtuosic p..."
1,train/00_BN2-166-Ab_solo_mic.wav,solo,"High-quality acoustic guitar solo, virtuosic p..."
2,train/00_Jazz2-187-F#_solo_mic.wav,solo,"High-quality acoustic guitar solo, virtuosic p..."
3,train/00_Funk3-98-A_solo_mic.wav,solo,"High-quality acoustic guitar solo, virtuosic p..."
4,train/00_Funk1-114-Ab_solo_mic.wav,solo,"High-quality acoustic guitar solo, virtuosic p..."
...,...,...,...
355,train/05_BN3-119-G_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
356,train/05_SS3-84-Bb_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
357,train/05_SS2-88-F_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
358,train/05_SS2-107-Ab_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."


In [ ]:
# create eval set
train, eval = train_test_split(all_df, test_size=0.1, random_state=42, stratify=all_df['audio_type'])
eval_list = eval['file_name'].tolist()
all_df.loc[all_df['file_name'].isin(eval_list), 'file_name'] = all_df.loc[all_df['file_name'].isin(eval_list), 'file_name'].str.replace('train', 'eval')
len(all_df[all_df['file_name'].str.contains('eval')])

36

In [ ]:
# move files into eval folder
os.makedirs(os.path.join(destination_dir, "eval"), exist_ok=True)
for file_name in eval_list:
    source_path = os.path.join(destination_dir, "train", os.path.basename(file_name))
    destination_path = os.path.join(destination_dir, "eval", os.path.basename(file_name))
    shutil.move(source_path, destination_path)

## Process Other Audio Dataset

### Chunk Data

In [ ]:
!pip install librosa soundfile

In [ ]:
dataset_path = "/content/drive/MyDrive/GuitarJam/audio_mono-mic"
solo_files = glob.glob(dataset_path + "/solo/*.wav")
comp_files = glob.glob(dataset_path + "/comp/*.wav")
print(len(solo_files), len(comp_files))
print(solo_files, comp_files)


In [ ]:
import os
import librosa
import soundfile as sf
sc, sr = librosa.load("/content/drive/MyDrive/GuitarJam/yt_music_dataset_2/train/isn_t_she_lovely_stevie_wonder_guitar_cover_dr__2.wav", sr=32000)
start = sr*13
split = sr*10
sc = sc[start:start+split]
sf.write(os.path.join("/content/drive/MyDrive/GuitarJam", "yt_music_demo.wav"), sc, sr)

In [ ]:
from IPython.display import Audio
Audio("/content/drive/MyDrive/GuitarJam/yt_music_dataset_2/train/isn_t_she_lovely_stevie_wonder_guitar_cover_dr__2.wav")

In [ ]:
Audio("/content/drive/MyDrive/GuitarJam/yt_music_demo.wav")

In [ ]:
import librosa
import soundfile as sf

comp_path = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp/train"
comp_files = glob.glob(comp_path + "/*.wav")

segments_path = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments"
if not os.path.exists(segments_path):
  os.mkdir(segments_path)

for f in comp_files:
  audio, sr = librosa.load(f, sr=None)
  split_point = sr * 15
  first_part = audio[:split_point]
  second_part = audio[split_point:]

  sf.write(os.path.join(segments_path, f.split('/')[-1][:-4]+"_first.wav"), first_part, sr)
  sf.write(os.path.join(segments_path, f.split('/')[-1][:-4]+"_last.wav"), second_part, sr)

segments_file = glob.glob(segments_path + "/*.wav")
print(len(segments_file))
print(segments_file)

In [ ]:
# train, eval split
audio_files = glob.glob("/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments/*.wav")
train_files, val_files = train_test_split(audio_files, test_size=0.1, random_state=42)

train_set = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments/train"
if not os.path.exists(train_set):
  os.mkdir(train_set)
else:
  shutil.rmtree(train_set)
for t in train_files:
  shutil.move(t, train_set+"/"+t.split('/')[-1])

val_set = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments/validation"
if not os.path.exists(val_set):
  os.mkdir(val_set)
else:
  shutil.rmtree(val_set)
for v in val_files:
  shutil.move(v, val_set+"/"+v.split('/')[-1])


In [ ]:
import glob, shutil, os
import pandas as pd
import re

train_set = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments/train"
val_set = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments/validation"

def key_to_number(key):
    key_mapping = {
        "C": 0, "C#": 1, "Db": 1,
        "D": 2, "D#": 3, "Eb": 3,
        "E": 4,
        "F": 5, "F#": 6, "Gb": 6,
        "G": 7, "G#": 8, "Ab": 8,
        "A": 9, "A#": 10, "Bb": 10,
        "B": 11
    }

    return str(key_mapping.get(key, -1))

df = pd.DataFrame(columns=['file_name', 'style', 'tempo', 'key', 'key_number'])
for t in glob.glob(train_set+"/*.wav"):
  name = t.split('/')[-1]
  labels = name.split('_')[1].split('-')
  df.loc[len(df)] = ["train/"+name, re.sub(r'\d', '', labels[0]), labels[1], labels[2], key_to_number(labels[2])]
for t in glob.glob(val_set+"/*.wav"):
  name = t.split('/')[-1]
  labels = name.split('_')[1].split('-')
  df.loc[len(df)] = ["validation/"+name, re.sub(r'\d', '', labels[0]), labels[1], labels[2], key_to_number(labels[2])]

df["prompt"] = "guitar, "+df["style"]+", tempo:"+df["tempo"]+", key:"+df["key_number"]
df.to_csv("/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp_segments/metadata.csv")

### prepare eval set for comp

In [ ]:
# create eval set
_, comp_eval = train_test_split(comp_df, test_size=0.1, random_state=42)
comp_eval_list = comp_eval['file_name'].tolist()

comp_df.loc[comp_df['file_name'].isin(comp_eval_list), 'file_name'] = comp_df.loc[comp_df['file_name'].isin(comp_eval_list), 'file_name'].str.replace('train', 'validation')
print("eval cnt", len(comp_df[comp_df['file_name'].str.contains('validation')]))

# move files into eval folder
comp_dir = "/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp"
os.makedirs(os.path.join(comp_dir, "validation"), exist_ok=True)
for file_name in comp_eval_list:
    source_path = os.path.join(comp_dir, "train", os.path.basename(file_name))
    destination_path = os.path.join(comp_dir, "validation", os.path.basename(file_name))
    shutil.move(source_path, destination_path)

eval cnt 18


In [ ]:
comp_df['prompt'] = comp_df['audio_type'].map(prompt_dict)
comp_df.to_csv("/content/drive/MyDrive/GuitarJam/audio_mono-mic/comp/metadata.csv", index=False)
comp_df.head()

,file_name,audio_type,prompt
0,train/00_Jazz3-150-C_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
1,train/00_Funk2-119-G_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
2,train/00_BN2-166-Ab_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
3,train/00_BN1-129-Eb_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."
4,train/00_BN2-131-B_comp_mic.wav,accompaniment,"High-quality acoustic guitar accompaniment, vi..."


In [ ]:
# !unzip /content/drive/MyDrive/GuitarJam/MusicGen/model_gl_ca_r32.zip -d /content/drive/MyDrive/GuitarJam/MusicGen/model_gl_ca_r32
# !mv /content/drive/MyDrive/GuitarJam/MusicGen/model_gl_ca_r32_zip/100_epoch_r_32model_gl_ca /content/drive/MyDrive/GuitarJam/MusicGen/model_gl_ca_r32
# !rm -rf /content/drive/MyDrive/GuitarJam/MusicGen/model_gl_ca_r32_zip


## Process YT Audio Dataset

In [ ]:
yt_df = pd.read_csv("/content/drive/MyDrive/GuitarJam/yt_music_dataset/metadata.csv")
yt_df.head()

,file_name,prompt
0,train/(full_video)_the_best_jam_i’ve_ever_witn...,"Music with Guitar, Guitar, Guitar - guitar jam..."
1,train/(full_video)_the_best_jam_i’ve_ever_witn...,"Music with Guitar, Guitar, Guitar - guitar jam..."
2,train/(full_video)_the_best_jam_i’ve_ever_witn...,"Music with Guitar, Guitar, Guitar - guitar jam..."
3,train/(full_video)_the_best_jam_i’ve_ever_witn...,"Music with Guitar, Guitar, Guitar - guitar jam..."
4,train/(full_video)_the_best_jam_i’ve_ever_witn...,"Music with Guitar, Guitar, Guitar - guitar jam..."


### Rename File Name

In [ ]:
def reformat_string(input_string: str) -> str:
    """
    Reformat a string based on the rules:
    1. Replace all punctuation with an underscore.
    2. Remove emojis, spaces, and numbers.
    3. Replace consecutive underscores with a single underscore.
    4. Strip punctuations from the start and end of the filename.
    The '__xx' postfix remains unchanged.

    :param input_string: The input string to process.
    :return: Reformatted string.
    """
    # Split the string into filename and suffix
    match = re.match(r'^(.*)(__[^_]+)$', input_string)
    if not match:
        return input_string  # Return the original string if it doesn't match the format

    filename, suffix = match.groups()

    # Replace all punctuation with an underscore
    filename = re.sub(r'[^\w\s]', '_', filename)

    # Remove emojis, spaces, and numbers
    filename = re.sub(r'[0-9]', '', filename)         # Remove numbers
    filename = re.sub(r'\s+', '', filename)          # Remove spaces
    filename = re.sub(r'[\U00010000-\U0010FFFF]', '', filename)  # Remove emojis

    # Replace consecutive underscores with a single underscore
    filename = re.sub(r'_+', '_', filename)

    # Strip leading and trailing punctuation (now underscores)
    filename = filename.strip('_')

    return f"{filename}{suffix}"


In [ ]:
filenames = [os.path.basename(f) for f in yt_df['file_name'].tolist()]
filename_map = {
    f: reformat_string(f) for f in filenames
}

new_yt_df = yt_df.copy()
new_yt_df['file_name'] = new_yt_df['file_name'].apply(lambda x: f"train/{filename_map[os.path.basename(x)]}")
new_yt_df

yt_data_dir = "/content/drive/MyDrive/GuitarJam/yt_music_dataset/train"
destination_dir = "/content/drive/MyDrive/GuitarJam/yt_music_dataset_2/train"
for k, v in filename_map.items():
    shutil.copy(os.path.join(yt_data_dir, k), os.path.join(destination_dir, v))


In [ ]:
# new_yt_df.to_csv("/content/drive/MyDrive/GuitarJam/yt_music_dataset_2/metadata.csv", index=False)
new_yt_df = pd.read_csv("/content/drive/MyDrive/GuitarJam/yt_music_dataset_2/metadata.csv")
new_yt_df

,file_name,prompt
0,train/full_video_the_best_jam_i_ve_ever_witnes...,"Music with Guitar, Guitar, Guitar - guitar jam..."
1,train/full_video_the_best_jam_i_ve_ever_witnes...,"Music with Guitar, Guitar, Guitar - guitar jam..."
2,train/full_video_the_best_jam_i_ve_ever_witnes...,"Music with Guitar, Guitar, Guitar - guitar jam..."
3,train/full_video_the_best_jam_i_ve_ever_witnes...,"Music with Guitar, Guitar, Guitar - guitar jam..."
4,train/full_video_the_best_jam_i_ve_ever_witnes...,"Music with Guitar, Guitar, Guitar - guitar jam..."
...,...,...
591,train/fingers_guitar_rhapsody_full_concert_fin...,"Music with Guitar, Guitar, Guitar - guitar liv..."
592,train/fingers_guitar_rhapsody_full_concert_fin...,"Music with Guitar, Guitar, Guitar - guitar liv..."
593,train/fingers_guitar_rhapsody_full_concert_fin...,"Music with Guitar, Guitar, Guitar - guitar liv..."
594,train/fingers_guitar_rhapsody_full_concert_fin...,"Music with Guitar, Guitar, Guitar - guitar liv..."


In [ ]:
# create eval set
_, yt_eval = train_test_split(new_yt_df, test_size=0.1, random_state=42)
yt_eval_list = yt_eval['file_name'].tolist()

new_yt_df.loc[new_yt_df['file_name'].isin(yt_eval_list), 'file_name'] = new_yt_df.loc[new_yt_df['file_name'].isin(yt_eval_list), 'file_name'].str.replace('train', 'validation')
print("eval cnt", len(new_yt_df[new_yt_df['file_name'].str.contains('validation')]))

# move files into eval folder
yt_dir = "/content/drive/MyDrive/GuitarJam/yt_music_dataset_2"
os.makedirs(os.path.join(yt_dir, "validation"), exist_ok=True)
for file_name in yt_eval_list:
    source_path = os.path.join(yt_dir, "train", os.path.basename(file_name))
    destination_path = os.path.join(yt_dir, "validation", os.path.basename(file_name))
    shutil.move(source_path, destination_path)

eval cnt 63
